Copyright © 2021, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# HMEQ Dataset : Build and Import Trained H2O.ai Models into SAS Model Manager

This notebook provides an example of how to build and train a Python model and then import the model into SAS Model Manager using the HMEQ data set. Lines of code that must be modified by the user, such as directory paths are noted with the comment "_Changes required by user._".

_**Note:** If you download only this notebook and not the rest of the repository, you must also download the hmeq.csv file from the data folder in the examples directory. These files are used when executing this notebook example._

Here are the steps shown in this notebook:

1. Import and review data and preprocess for model training.
2. Build, train, and access an H2O.ai generalized linear estimator model.
3. Serialize the model into pickle or MOJO files.
4. Write the metadata JSON files needed for importing into SAS Model Manager.
4. Write a score code Python file for model scoring.
5. Zip the model, JSON, and score code files into an archive file.
6. Import the ZIP archive file to SAS Model Manager via the Session object and relevant function call.

### Python Package Imports

In [1]:
# Standard Library
from pathlib import Path
import warnings

# Third Party
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h2o
import pprint as pp
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# Application Specific
import sasctl.pzmm as pzmm
from sasctl import Session, publish_model

In [2]:
# Global Package Options
pd.options.mode.chained_assignment = None  # default="warn"
plt.rc("font", size=14)
# Ignore warnings from pandas about SWAT using a feature that will be depreciated soon
warnings.simplefilter(action="ignore", category=FutureWarning)

In [3]:
h2o.__version__

'3.40.0.3'

On SAS Viya, models created in H2O versions 3.24 and under are only compatible in the binary model format. For H2O versions 3.26+, models can be in the MOJO or binary model format. If using a binary model, the H2O version on the SAS Viya server must match the exact version of H2O used to create the model.

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21); OpenJDK 64-Bit Server VM JBR-11.0.13.7-1751.21-jcef (build 11.0.13+7-b1751.21, mixed mode)
  Starting server from /Users/dalmoo/opt/anaconda3/envs/yeehaw/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/vs/np2dp7cs1y7ggk5pl92q_rb40000gn/T/tmp0_92875_
  JVM stdout: /var/folders/vs/np2dp7cs1y7ggk5pl92q_rb40000gn/T/tmp0_92875_/h2o_dalmoo_started_from_python.out
  JVM stderr: /var/folders/vs/np2dp7cs1y7ggk5pl92q_rb40000gn/T/tmp0_92875_/h2o_dalmoo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,6 months and 1 day
H2O_cluster_name:,H2O_from_python_dalmoo_6awy1u
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


### Import and Review Data Set

In [5]:
hmeq_data = h2o.import_file("data/hmeq.csv", sep= ",")
hmeq_data.shape

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


(5960, 13)

### Preprocess Data

In [6]:
hmeq_data["BAD"] = hmeq_data["BAD"].asfactor()

train, validation, test = hmeq_data.split_frame(ratios=[.6, .2], seed=42)

y = "BAD"
x = list(hmeq_data.columns)
x.remove(y)

### Create, Train, and Assess Model

In [7]:
glm = H2OGeneralizedLinearEstimator(family="binomial", model_id="glmfit", lambda_search=True)
glm.train(x=x, y=y, training_frame=train, validation_frame=validation)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: glmfit


GLM Model: summary
    family    link    regularization                                 lambda_search                                                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 9.244E-4 )  nlambda = 100, lambda.max = 0.2455, lambda.min = 9.244E-4, lambda.1se = -1.0  18                            17                             93                      py_3_sid_a269

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.12410463474559494
RMSE: 0.3522848772592927
LogLoss: 0.40208197287665776
AUC: 0.7849415147958837
AUCPR: 0.5691248967099318
Gini: 0.5698830295917674
Null degrees of freedom: 3580
Residual degrees of freedom: 3563
Null deviance: 3597.150438148379
Residual deviance: 2879.711089742623
AIC: 2915.711089742623

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2574171649920362
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      2446  414  0.1448   (414.0/2860.0)
1      314   407  0.4355   (314.0/721.0)
Total  2760  821  0.2033   (728.0/3581.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.257417     0.527886  207
max f2                       0.152297     0.626112  279
max f0point5                 0.388606     0.557126  144
max accuracy                 0.567259     0.835242  90
max precision                0.998809     1         0
max recall                   0.00123775   1         399
max specificity              0.998809     1         0
max absolute_mcc             0.266185     0.402098  203
max min_per_class_accuracy   0.17828      0.706643  258
max mean_per_class_accuracy  0.214315     0.719503  234
max tns                      0.998809     2860      0
max fns                      0.998809     717       0
max fps                      0.00626272   2860      398
max tps                      0.00123775   721       399
max tnr                      0.998809     1         0
max fnr                      0.998809     0.994452  0
max fpr                      0.00626272   1         398
max tpr                      0.00123775   1         399

Gains/Lift Table: Avg response rate: 20.13 %, avg score: 20.13 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100531                   0.946682           4.96671   4.96671            1                0.980507   1                           0.980507            0.0499307       0.0499307                  396.671   396.671            0.0499307
2        0.0201061                   0.85234            4.82875   4.89773            0.972222         0.898794   0.986111                    0.93965             0.0485437       0.0984743                  382.875   389.773            0.0981247
3        0.0301592                   0.747956           3.863     4.55282            0.777778         0.798407   0.916667                    0.892569            0.038835        0.137309                   286.3     355.282            0.134162
4        0.04

In [8]:
# Check the model performance and print its accuracy
glm_performance = glm.model_performance(test)
print(glm_performance.accuracy())

[[0.5513512979207219, 0.8486897717666948]]


In [ ]:
glm.pre

### Register Models in SAS Model Manager

In [9]:
model_prefix = "glmfit"
binary_folder = Path.cwd() / "data/hmeqModels/H2OBinaryGLM/" # Changes needed by user
mojo_folder = Path.cwd() / "data/hmeqModels/H2OMOJOGLM/" # Changes needed by user

In [10]:
# Save the model as a H2O binary model file
pzmm.PickleModel.pickle_trained_model(
    model_prefix=model_prefix + "_binary",
    trained_model=glm, 
    pickle_path=binary_folder, 
    is_h2o_model=True, 
    is_binary_model=True
)

In [11]:
# Save the model as a H2O MOJO model file
pzmm.PickleModel.pickle_trained_model(
    model_prefix=model_prefix + "_mojo",
    trained_model=glm,
    pickle_path=mojo_folder, 
    is_h2o_model=True
)

In [12]:
train_df = train.as_data_frame()
# Write input variable mapping to a json file
pzmm.JSONFiles.write_var_json(train_df[x], is_input=True, json_path=binary_folder)
pzmm.JSONFiles.write_var_json(train_df[x], is_input=True, json_path=mojo_folder)

# Set output variables and assign an event threshold, then write output variable mapping
output_var = pd.DataFrame(
    columns=["EM_CLASSIFICATION", "EM_EVENTPROBABILITY"],
    data=[["A", 0.5]]
)
pzmm.JSONFiles.write_var_json(output_var, is_input=False, json_path=binary_folder)
pzmm.JSONFiles.write_var_json(output_var, is_input=False, json_path=mojo_folder)

# Write model properties to a json file
pzmm.JSONFiles.write_model_properties_json(
    model_name=model_prefix + "_binary",
    model_desc="Binary H2O model.",
    target_variable=y,
    target_values=["0", "1"],
    json_path=binary_folder,
    modeler="sasdemo"
)
pzmm.JSONFiles.write_model_properties_json(
    model_name=model_prefix + "_mojo",
    model_desc="MOJO H2O model.",
    target_variable=y,
    target_values=["0", "1"],
    json_path=mojo_folder,
    modeler="sasdemo"
)

# Write model metadata to a json file
pzmm.JSONFiles.write_file_metadata_json(model_prefix=model_prefix + "_binary", json_path=binary_folder)
pzmm.JSONFiles.write_file_metadata_json(model_prefix=model_prefix + "_mojo", json_path=mojo_folder, is_h2o_model=True)

inputVar.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OBinaryGLM/inputVar.json
inputVar.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OMOJOGLM/inputVar.json
outputVar.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OBinaryGLM/outputVar.json
outputVar.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OMOJOGLM/outputVar.json
ModelProperties.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OBinaryGLM/ModelProperties.json
ModelProperties.json was successfully written and saved to /Users/dalmoo/Documents/GitHub/python-sasctl/examples/data/hmeqModels/H2OMOJOGLM/ModelProperties.json
fileMetadata.json was successfully written and saved to /Users/dalmoo/Documents/

In [13]:
import getpass
username = getpass.getpass()
password = getpass.getpass()
host = "sas.demo.com"
sess = Session(host, username, password, protocol="http")

In [14]:
binary_model = pzmm.ImportModel.import_model(
    model_files=binary_folder, 
    model_prefix=model_prefix + "_binary",
    project="H2OModels", 
    input_data=train_df[x], 
    predict_method=[glm.predict, [list]], 
    binary_h2o_model=True, 
    score_metrics=["EM_CLASSIFICATION", "EM_EVENTPROBABILITY"],
    missing_values=True,
    overwrite_model=True,
    model_file_name="glmfit_binary.pickle"
)
pzmm.ScoreCode.score_code = ""

mojo_model = pzmm.ImportModel.import_model(
    model_files=mojo_folder, 
    model_prefix=model_prefix + "_mojo",
    project="H2OModels", 
    input_data=train_df[x], 
    predict_method=[glm.predict, [list]], 
    mojo_model=True, 
    score_metrics=["EM_CLASSIFICATION", "EM_EVENTPROBABILITY"],
    missing_values=True,
    overwrite_model=True,
    model_file_name="glmfit_mojo.mojo"
)

All model files were zipped to ~\examples\data\hmeqModels\H2OBinaryGLM.
Model was successfully imported into SAS Model Manager as glmfit_binary with the following UUID: 5929a748-f9b2-4285-b73a-45c40659b4b0.
Model score code was written successfully to ~\examples\data\hmeqModels\H2OBinaryGLM\score_glmfit_binary.py and uploaded to SAS Model Manager.
All model files were zipped to ~\examples\data\hmeqModels\H2OMOJOGLM.
Model was successfully imported into SAS Model Manager as glmfit_mojo with the following UUID: 4c5dd027-d442-4860-9e96-9c26060dc727.
Model score code was written successfully to ~\examples\data\hmeqModels\H2OMOJOGLM\score_glmfit_mojo.py and uploaded to SAS Model Manager.


### Run a Score Test in SAS Model Manager

In [15]:
# Publish the model to the SAS Microanalytic Score destination in SAS Model Manager
module = publish_model(mojo_model[0], "maslocal", name="HMEQMOJO_publish", replace=True)

In [16]:
# Instantiate a API call logger to visualize score calls in realtime
sess.add_stderr_logger(level=20)

<StreamHandler stderr (INFO)>

In [17]:
# Convert h2o dataframe to pandas dataframe
X = train[0:10,:].as_data_frame()
result = []
# Step through the rows of data and collect the score from SAS Microanalytic Score publish destination
for index, row in X.iterrows():
    result.append(module.score(row))

HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 POST http://demo.sas.com/microanalyticScore/modules/hmeqmojo_publish/steps/score
HTTP/1.1 201 ht

In [18]:
# Scoring results
pp.pprint(result)

[(0.0, '0', 0.9470777188580571),
 (0.0, '0', 0.8697221847625589),
 (0.0, '0', 0.932352798571864),
 (0.0, '0', 0.9579361586939033),
 (0.0, '0', 0.8422587334478235),
 (0.0, '0', 0.9112367410859918),
 (0.0, '0', 0.7222737872462567),
 (0.0, '0', 0.8016194612045261),
 (0.0, '0', 0.9285316160726795),
 (0.0, '0', 0.9619797616826327)]
